In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import utils
import tensorboard_log_pixel

import models

import torchbearer as tb

from tensorboard_log_pixel import ReconstructionsLogger, TensorBoardModelLogger, LatentSpaceReconLogger, RandomReconLogger
from torchbearer import Trial
from torchbearer.callbacks.tensor_board import TensorBoard
from utils import AEDatasetWrapper

import warnings
# warnings.filterwarnings('ignore')

from apex import amp

In [2]:
# torch.backends.cudnn.enabled=False

# Parameters:
params = {'batch_size': 32,
            'nEpoch': 50,
            'imgSize': 64,
            'zsize': 64,
            'depth': 0,
             'outcn': 0,
            'lr': 1e-3,
             'trainSize': 1500,
             'testSize': 150}

# Dataset construction
transform = transforms.Compose([
        transforms.ToTensor(),  # convert to tensor
    ])

In [3]:
# trainset = AEDatasetWrapper(torchvision.datasets.CIFAR10('.', train=True, transform=transform))
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=params['batch_size'], shuffle=True, num_workers=3)
# testset = AEDatasetWrapper(torchvision.datasets.CIFAR10('.', train=False, transform=transform))
# testloader = torch.utils.data.DataLoader(testset, batch_size=params['batch_size'], shuffle=False, num_workers=3)
trainloader, testloader = utils.getloaders('imagenet_valid/', batch_size=params['batch_size'])

In [4]:
# construct the encoder, decoder and optimiser
iSize = params['imgSize']

vae = models.PixelVAE(iSize, params['zsize'], params['depth'], params['outcn'])
optimizer = optim.Adam(vae.parameters(), lr=params['lr'])

tb_comment = 'testing'
tbl = TensorBoard(write_graph=True, comment=tb_comment)
tbml = TensorBoardModelLogger(comment=tb_comment)
rsl = ReconstructionsLogger(comment=tb_comment, output_shape=(3, iSize, iSize))
rrsl = tensorboard_log_pixel.RandomPixReconLogger(comment=tb_comment, latent_dim=params['zsize'], output_shape=(3, iSize, iSize))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ...

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
trial = Trial(vae, optimizer, models.PixelVAE.loss_ce, metrics=['loss', models.sampled_recons_loss()], callbacks=[tbl, rsl, tbml, rrsl]).to(device)
trial.with_generators(trainloader, val_generator=testloader)

--------------------- OPTIMZER ---------------------
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

-------------------- CRITERION ---------------------
<function PixelVAE.loss_ce at 0x7ff011fcf158>

--------------------- METRICS ----------------------
['loss', 'recons_loss']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.tensor_board.TensorBoard', 'tensorboard_log_pixel.ReconstructionsLogger', 'tensorboard_log_pixel.TensorBoardModelLogger', 'tensorboard_log_pixel.RandomPixReconLogger']

---------------------- MODEL -----------------------
PixelVAE(
  (enc): ImEncoder(
    (encoder): Sequential(
      (0): Block(
        (upchannels): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        (seq): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
          (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(

In [7]:
amp.initialize(vae, optimizer, opt_level="O1")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


(PixelVAE(
   (enc): ImEncoder(
     (encoder): Sequential(
       (0): Block(
         (upchannels): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
         (seq): Sequential(
           (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (1): ReLU()
           (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (3): ReLU()
           (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (5): ReLU()
         )
       )
       (1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
       (2): Block(
         (upchannels): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
         (seq): Sequential(
           (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (1): ReLU()
           (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (3): ReLU()
           (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(

In [ ]:
utils.limit_steps(trial, params['trainSize'], params['testSize'])
trial.run(20)